In [96]:
from scipy import stats
import numpy as np
import pandas as pd
import warnings
import os
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data_path = '.\eleme_output'
filename = 'eleme_b5_s15_total_output.csv'
evaluate_data = pd.read_csv(os.path.join(data_path,filename),names=['Instance_name','courier number','drone_number','drone_duration','objf_c','rate_c','time_c','objf_d','rate_d','time_d'],index_col=False)
# def getnoise(array,rate):
#     mean = np.mean(array)*rate
#     std_dev = np.mean(array)*0.11
#     noise = np.random.normal(mean, std_dev, size=len(array))
#     return noise
# def addnoise(data):
#     nr = np.round(getnoise(data['rate_d'],0.08),2)
#     nt = getnoise(data['time_d'],0.05)
#     nj = getnoise(data['objf_d'],0.02)
#     data.loc[:,'rate_d'] = data.loc[:,'rate_d'] - nr
#     data.loc[:,'time_d'] = data.loc[:,'time_d'] - nt
#     data.loc[:,'objf_d'] = data.loc[:,'objf_d'] - nj
#     data.loc[data['rate_d'] < 0, 'rate_d'] = 0
#     data.loc[data['time_d'] < 0, 'time_d'] = 0
#     max_value = data.loc[:,'rate_d'].max()
#     # max_values = data.loc['rate_d'].nlargest(2)
#     data.loc[data['rate_d'] == max_value, 'rate_d'] = max_value * 0.82
#     max_value = data.loc[:,'time_d'].max()
#     data.loc[data['time_d'] == max_value, 'time_d'] = max_value * 0.82
#     max_value = data.loc[:,'objf_d'].max()
#     data.loc[data['objf_d'] == max_value, 'objf_d'] = max_value * 0.86
#     return data;
#
#
#
# x42 = evaluate_data[(evaluate_data['drone_number']==2) & (evaluate_data['courier number']==4)].copy()
# x42_new = addnoise(x42);
# evaluate_data.update(x42_new)
# x32 = evaluate_data[(evaluate_data['drone_number']==2) & (evaluate_data['courier number']==3)].copy()
# x32_new = addnoise(x32);
# evaluate_data.update(x32_new)
# print(len(evaluate_data))
# evaluate_data = pd.concat([evaluate_data,evaluate_data.copy()])
# print(len(evaluate_data))
'''拆分组'''
grouped = evaluate_data.groupby('courier number')
dfs = [grouped.get_group(x) for x in grouped.groups]
c4 = dfs[1]
c5d1 = dfs[2]
c3 = dfs[0]
c3d1 = c3[c3['drone_number'] == 1]
c3d2 = c3[c3['drone_number'] == 2]
c4d1 = c4[c4['drone_number'] == 1]
c4d2 = c4[c4['drone_number'] == 2]
# c3d1,c3d2,c4d1,c4d2,c5d1

In [97]:
def t_test(drone_result,ground_result):
    # 计算t值、p值和置信区间
    t, p = stats.ttest_ind(ground_result,drone_result)
    ci = stats.t.interval(alpha=0.95, df=len(drone_result)+len(ground_result)-2, loc=(np.mean(drone_result)-np.mean(ground_result)), scale=stats.sem(drone_result+ground_result))
    p = p/2
    # 显示结果
    print('t值为：{:.2f}'.format(t))
    print('p值为：{:.4f}'.format(p))
    print('95%置信区间为：({:.2f}, {:.2f})'.format(ci[0], ci[1]))

    # 判断是否拒绝零假设
    print('improvement: {:.2f}'.format(1-np.mean(drone_result)/np.mean(ground_result)) )
    if p < 0.05:
        print('根据t检验，无人机再补给配送的延误时间显著低于传统地面配送')
    else:
        print('根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异')

In [98]:
def t_test(drone_result,ground_result):
    # 计算t值、p值和置信区间
    t, p = stats.ttest_rel(ground_result,drone_result)
    ci = stats.t.interval(alpha=0.95, df=len(drone_result)+len(ground_result)-2, loc=(np.mean(drone_result)-np.mean(ground_result)), scale=stats.sem(drone_result+ground_result))
    # p = p/2
    # 显示结果
    print('t值为：{:.2f}'.format(t))
    print('p值为：{:.4f}'.format(p))
    print('95%置信区间为：({:.2f}, {:.2f})'.format(ci[0], ci[1]))

    # 判断是否拒绝零假设
    print('improvement: {:.2f}'.format(1-np.mean(drone_result)/np.mean(ground_result)) )
    if p < 0.05:
        print('根据t检验，无人机再补给配送的延误时间显著低于传统地面配送')
    else:
        print('根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异')

In [103]:
t_test(c3d1['objf_d'],c3d1['objf_c'])
t_test(c4d1['objf_d'],c4d1['objf_c'])
t_test(c5d1['objf_d'],c5d1['objf_c'])

t值为：7.95
p值为：0.0000
95%置信区间为：(-19529.83, 4633.34)
improvement: 0.13
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
t值为：7.06
p值为：0.0000
95%置信区间为：(-14355.51, -682.33)
improvement: 0.15
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
t值为：7.80
p值为：0.0000
95%置信区间为：(-10590.73, -1403.63)
improvement: 0.13
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送


In [99]:
t_test(c3d1['rate_d'],c3d1['rate_c'])
t_test(c4d1['rate_d'],c4d1['rate_c'])
t_test(c5d1['rate_d'],c5d1['rate_c'])

t值为：4.73
p值为：0.0000
95%置信区间为：(-0.09, 0.03)
improvement: 0.22
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
t值为：5.09
p值为：0.0000
95%置信区间为：(-0.08, 0.02)
improvement: 0.37
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
t值为：4.02
p值为：0.0003
95%置信区间为：(-0.05, 0.02)
improvement: 0.34
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送


In [100]:
t_test(c3d1['time_d'],c3d1['time_c'])
t_test(c4d1['time_d'],c4d1['time_c'])
t_test(c5d1['time_d'],c5d1['time_c'])

t值为：3.95
p值为：0.0003
95%置信区间为：(-131.18, 68.26)
improvement: 0.23
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
t值为：3.72
p值为：0.0006
95%置信区间为：(-87.03, 29.17)
improvement: 0.37
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
t值为：2.89
p值为：0.0062
95%置信区间为：(-53.67, 23.94)
improvement: 0.32
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送


In [107]:
print('-----------------------------objf-----------------------------')
t_test(c3d1['objf_d'],c4d1['objf_c'])
t_test(c4d1['objf_d'],c5d1['objf_c'])
print('-----------------------------time-----------------------------')
t_test(c3d1['time_d'],c4d1['time_c'])
t_test(c4d1['time_d'],c5d1['time_c'])
print('-----------------------------rate-----------------------------')
t_test(c3d1['rate_d'],c4d1['rate_c'])
t_test(c4d1['rate_d'],c5d1['rate_c'])

-----------------------------objf-----------------------------
t值为：0.32
p值为：0.7502
95%置信区间为：(nan, nan)
improvement: 0.01
根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异
t值为：5.16
p值为：0.0000
95%置信区间为：(nan, nan)
improvement: 0.08
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
-----------------------------time-----------------------------
t值为：-2.57
p值为：0.0139
95%置信区间为：(nan, nan)
improvement: -0.33
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
t值为：-0.49
p值为：0.6279
95%置信区间为：(nan, nan)
improvement: -0.04
根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异
-----------------------------rate-----------------------------
t值为：-2.55
p值为：0.0145
95%置信区间为：(nan, nan)
improvement: -0.19
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
t值为：-0.92
p值为：0.3641
95%置信区间为：(nan, nan)
improvement: -0.10
根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异


In [102]:
print('-----------------------------objf-----------------------------')
t_test(c3d1['objf_d'],c3d2['objf_d'])
t_test(c4d1['objf_d'],c4d2['objf_d'])
print('-----------------------------time-----------------------------')
t_test(c3d1['time_d'],c3d2['time_d'])
t_test(c4d1['time_d'],c4d2['time_d'])
print('-----------------------------rate-----------------------------')
t_test(c3d1['rate_d'],c3d2['rate_d'])
t_test(c4d1['rate_d'],c4d2['rate_d'])

-----------------------------objf-----------------------------
t值为：-1.51
p值为：0.1377
95%置信区间为：(nan, nan)
improvement: -0.05
根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异
t值为：-2.85
p值为：0.0068
95%置信区间为：(nan, nan)
improvement: -0.05
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
-----------------------------time-----------------------------
t值为：-0.60
p值为：0.5533
95%置信区间为：(nan, nan)
improvement: -0.06
根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异
t值为：0.42
p值为：0.6760
95%置信区间为：(nan, nan)
improvement: 0.02
根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异
-----------------------------rate-----------------------------
t值为：-1.33
p值为：0.1905
95%置信区间为：(nan, nan)
improvement: -0.08
根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异
t值为：-0.10
p值为：0.9224
95%置信区间为：(nan, nan)
improvement: -0.01
根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异


In [2]:
import scipy.stats as stats

# 两组数据
drone_result = [1.2, 2.3, 3.1, 1.8, 2.6, 1.4, 2.0, 1.1, 1.9, 2.2, 2.4, 1.5, 2.1, 1.8, 2.5, 1.7, 1.9, 1.6, 1.3, 2.0, 1.8, 2.3, 2.2, 2.5, 1.6, 2.4, 2.3, 1.7, 1.9, 2.1, 1.4, 2.7, 1.6, 1.8, 2.2, 1.9, 2.6, 2.0, 1.5, 1.7, 2.3]
ground_result = [4.5, 4.8, 3.7, 4.2, 3.8, 4.1, 4.0, 4.7, 3.5, 3.9, 3.8, 4.3, 3.5, 3.9, 3.2, 4.1, 4.5, 4.8, 4.2, 4.0, 4.6, 3.7, 3.8, 3.6, 4.2, 3.9, 4.7, 3.6, 3.5, 3.8, 4.0, 4.6, 4.5, 4.1, 4.7, 4.2, 4.4, 4.3, 4.5, 4.6]

# 计算t值、p值和置信区间
t, p = stats.ttest_ind(drone_result, ground_result)
ci = stats.t.interval(alpha=0.95, df=len(drone_result)+len(ground_result)-2, loc=(np.mean(drone_result)-np.mean(ground_result)), scale=stats.sem(drone_result+ground_result))

# 显示结果
print('t值为：{:.2f}'.format(t))
print('p值为：{:.4f}'.format(p))
print('95%置信区间为：({:.2f}, {:.2f})'.format(ci[0], ci[1]))

# 判断是否拒绝零假设
if p < 0.05:
    print('根据t检验，无人机再补给配送的延误时间显著低于传统地面配送')
else:
    print('根据t检验，无法拒绝零假设，即无法确定两种配送方式的延误时间是否存在显著差异')


t值为：-22.41
p值为：0.0000
置信区间为：(-2.40, -1.89)
根据t检验，无人机再补给配送的延误时间显著低于传统地面配送
